In [1]:
%%capture

%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tfl_training_sbi

In [2]:
%presentation_style

In [3]:
%%capture

%set_random_seed 12

In [4]:
%load_latex_macros


$\newcommand{\vect}[1]{{\mathbf{\boldsymbol{#1}} }}$
$\newcommand{\amax}{{\text{argmax}}}$
$\newcommand{\P}{{\mathbb{P}}}$
$\newcommand{\E}{{\mathbb{E}}}$
$\newcommand{\R}{{\mathbb{R}}}$
$\newcommand{\Z}{{\mathbb{Z}}}$
$\newcommand{\N}{{\mathbb{N}}}$
$\newcommand{\C}{{\mathbb{C}}}$
$\newcommand{\abs}[1]{{ \left| #1 \right| }}$
$\newcommand{\simpl}[1]{{\Delta^{#1} }}$


<img src="_static/images/aai-institute-cover.png" alt="Snow" style="width:100%;">
<div class="md-slide title">Conditional Density Estimation</div>

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from tqdm.notebook import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

from tfl_training_sbi import mdn


# Conditional Density Estimation

_"In statistics, probability density estimation or simply density estimation is the construction of an **estimate, based on observed data**, of an unobservable underlying probability density function."_ (Wikipedia)

It plays a crucial role in various areas of machine learning, e.g. unsupervised learning, generative modeling, and probabilistic modeling. It allows us to gain insights into the underlying data distribution and enables us to perform various tasks such as outlier detection or data generation.

Here we want to use Density Estimation to **approximate a distribution over a free variable, given a set of observations**.

## Density Estimation 

Given samples $\theta_i,~i=1,\dots,n$ with $\theta_i \sim p(\theta)$ but $p(\theta)$ **unknown**.

Goal: find $\hat{p}(\theta) \approx p(\theta)$.
<figure>
  <center>
  <img src=_static/images/density_est_species.png style="width:45%"/>
    <figcaption>Fig. 1 - Plot from <a href"https://scikit-learn.org/stable/auto_examples/neighbors/plot_species_kde.html">scikit-learn</a></figcaption>
</figure>

## Methods for Density Estimation

- Histograms
- Kernel density estimation 
- Non-parametric approaches 
- Neural networks (VAEs, GANs, Normalizing Flows, ...)

## Example: Maximum Likelihood Estimation (MLE)

Given observations $\theta_i,~i=1,\dots,n$, find $\hat{p}(\theta)$

- Assumption: Data follows Normal distribution 
- Maximize Likelihood of $\theta_i$ under $\hat{p}$
- Find $\mu, \sigma$ where $\hat{p} := \mathcal{N}(\theta_i \mid \mu, \sigma)$ is maximized:

$$
\mu^{\prime}, \sigma^{\prime} = \underset{\mu, \sigma}{\operatorname{argmin}} - \sum_i^n \log \mathcal{N}(\theta_i; \mu, \sigma)
$$

<figure>
    <center>
    <img src="_static/images/density_est_toy_example_gauss.png" style="width:50%"/>
    <figcaption>Fig. 2 - Figure taken from <a href="https://mlcolab.org/simulation-based-inference-for-scientific-discovery">MLColab</a></figcaption>
</figure>

## Conditional Density Estimation

Above, we learned a marginal density $p(\theta)$. 

But often, we are interested in conditional density: 

What is the distribution of New York taxi drop-off locations $x$, given that we picked someone up at $\theta$? 

$$
p(\text{drop-off} \mid \text{pick-up}=\theta) = p(x \mid \theta)
$$

<figure>
    <center>
    <img src="_static/images/taxi.png" style="width:50%"/>
    <figcaption>Fig. 3 - Figure taken from <a href="https://mlcolab.org/simulation-based-inference-for-scientific-discovery">MLColab</a></figcaption>
</figure>

## Parametric Conditional Density Estimation

Given a fixed value $\theta$, parameterize a probability density over data $x$.
E.g. compute the conditional mean and variance of a Gaussian distribution using a function $f:\theta \mapsto \hat{\mu}, \hat{\sigma}$.

$$
p(x \mid \theta) = \mathcal{N}(x; (\mu, \sigma) = f(\theta))
$$

$f$ can be any function, e.g. a neural network.

Estimate the distribution parameters $\mu(\theta), \sigma(\theta)$ of

$$
p(x \mid \mu(\theta), \sigma(\theta)) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp(-\frac{(x-\mu(\theta))^2}{2\sigma^2(\theta)})
$$

by minimizing the negative log-Likelihood of the data under the model.

$$
\mathcal{L}(x, \theta) = - \sum_i \log \mathcal{N}(x_i; \mu(\theta_i), \sigma(\theta_i))
$$

Note, this is closely related to Bayesian linear regression.

## Example: Neural Conditional Density Estimation

**Setup**: Observations $x = \theta + 0.3\sin(2 \pi \theta) + \epsilon$ with $\theta
\sim \mathcal{U}(0,1)$ and $\epsilon \sim \mathcal{N}(0, 0.05)$.

**Goal**: Approximate $p(x \mid \theta = 0.1)$

**Method**: Use a neural network that predicts the parameters of the density model (e.g., a Gaussian). 


<figure>
    <center>
    <img src="_static/images/cde_gaussian.png" style="width:50%"/>
    <figcaption>Fig. 4 - Conditional density estimation with parametric Gaussian. Figure taken from <a href="https://mlcolab.org/simulation-based-inference-for-scientific-discovery">MLColab</a></figcaption>
</figure>

## Create  the Dataset 

In [ ]:
sample_size = 10_000
dim = 1

theta = torch.rand((sample_size, dim))
noise = torch.rand((sample_size, dim)) * 0.2

x = theta + 0.3 * torch.sin(2 * math.pi * theta) + noise


In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(theta[::15], x[::15], "go", alpha=0.2)
plt.axvline(0.1, color="black", linestyle="--")
plt.ylabel("$x$", fontsize=16)
plt.xlabel("$\\theta$", fontsize=16)
plt.show()


## Task: Train a neural network to estimate the parameters $\mu(\theta), \sigma(\theta)$ 

To do so, you have to create a torch Dataset and DataLoader containing `theta` and `x`. Further, you have to specify a small network with a suitable output dimension and an optimizer. Finally, you have to define the training loop in which the neg. log-Likelihood loss is minimized.

You can use the following structure to do so.

```python
dataset = # todo dataset of theta and x

train_loader = # todo: dataloader to the above dataset

# define a simple neural network to parameterize the conditional density
net = nn.Sequential(
    # todo: define model 
)

opt = # todo: define an optimizer 

# train the neural network for 100 epochs
for e in trange(100):
    for (
        theta_batch,
        x_batch,
    ) in train_loader:
        opt.zero_grad()
        nn_output = net(theta_batch)
        mean = nn_output[:, 0].unsqueeze(1)
        std = torch.exp(nn_output[:, 1]).unsqueeze(1)
        prob_Gauss = # todo: implement neg. log-Likelihood loss using assuming a Gaussian distr.
        loss = -torch.log(prob_Gauss).sum()
        loss.backward()
        opt.step()
```

In [ ]:
# Solution task 1.
# %load -r 7-36 ./solutions/solutions_nb_02.py

## Inspecting the Learned Distribution


In [ ]:
theta_cond = torch.tensor([0.1])
estimated_params = net(theta_cond)

cond_mean, cond_std = (
    estimated_params[0].detach().numpy(),
    torch.exp(estimated_params[1]).detach().numpy(),
)

print(f"Estimated mean: {cond_mean:.3f} and sigma: {cond_std:.3f}")


In [ ]:
plt.figure(figsize=(8, 4))

plt.plot(theta[::15], x[::15], "go", alpha=0.25)
plt.plot(
    [theta_cond[0], theta_cond[0]],
    [cond_mean - cond_std, cond_mean + cond_std],
    "k-",
    linewidth=2,
)
plt.scatter(theta_cond[0], cond_mean, c="k", s=60)

plt.ylabel("$x$", fontsize=16)
plt.xlabel("$\\theta$", fontsize=16)

plt.show()


## The Problem with this approach

- The conditional density estimator might be **too simple** to capture the true distribution
- Realistic distributions are more complex than a single Gaussian

## Test: Switch $\theta$ and $x$ and check what goes wrong

- yields $\theta = g(x)$
- $\hat{p}(\theta \mid x = 0.5) = \mathcal{N}(\theta; \mu(x)=0.44, \sigma(x)=0.22)$ 

In [ ]:
plt.figure(figsize=(8, 4))

plt.plot(x[::15], theta[::15], "go", alpha=0.25)
plt.plot(
    [0.5, 0.5],
    [0.44 - 0.22, 0.44 + 0.22],
    "k-",
    linewidth=2,
)
plt.scatter(0.5, 0.44, c="k", s=60)

plt.xlabel("$x$", fontsize=16)
plt.ylabel("$\\theta$", fontsize=16)

plt.show()


## Mixture Density Networks

Instead of learning to predict the mean and variance of a single Gaussian, a mixture density network (MDN) defines a linear combination of $k$ Gaussians. Each Gaussian distribution can be parameterized. 

\begin{align}
\mathcal{L} &= - \sum_{i=1} \log q(\theta_i \mid x_i) \\
 &= - \sum_{i=1} \log \sum_{j=1}^k \alpha_j \mathcal{N}(\theta_i; \mu_j(x_i), \sigma_j(x_i))
\end{align}

The MDN learns to predict mixture weights $\alpha_j$, means $\mu_j$ and (co-)variances $\sigma_j$, as a function of the input. 

## Train a Mixture Density Network

Use a MDN to parameterize a mixture of Gaussians to approximate the multi-modal distribution of the data shown above. 

In [ ]:
# prepare data set and data loader
# note, the x and theta are swapped compared to the previous example
dataset = TensorDataset(x, theta)
train_loader = DataLoader(dataset, batch_size=128)


**Task:** Define a Mixture Density Network using the `MultivariateGaussianMDN` class, provided by the `mdn` module. The object takes four parameters, which you have to define. 

1. `features`: number of free parameters 
2. `hidden_net`: a neural network used for feature extraction; this can be `None` but should be constructed in a way that important information can be extracted from the incoming data
3. `num_components`: number $k$ from above, i.e. the number of Gaussians in the linear combination 
4. `hidden_features`: vector size of the output from the feature extractor 

Feel free to use the following structure. 

```python
mixture_density_net = mdn.MultivariateGaussianMDN(
    features= ,         # todo: set the number of variables
    hidden_net= ,       # todo: define a feature extractor network
    num_components= ,   # todo: define the number of Gaussians 
    hidden_features= ,  # todo: define the output dimension of the feat. extractor 
)
```

In [ ]:
# Solution task 2.
# %load -r 39-58 ./solutions/solutions_nb_02.py

In [ ]:
# fit MDN using negative log-likelihood loss
opt = Adam(mixture_density_net.parameters(), lr=0.001)
num_epochs = 200
for e in trange(num_epochs):
    for x_batch, theta_batch in train_loader:
        opt.zero_grad()
        
        # Get log probability of theta under the MDN, given x.
        log_probs = mixture_density_net.log_prob(theta_batch, context=x_batch)

        # Take negative log prob as loss.
        loss = -log_probs.sum()
        loss.backward()
        opt.step()


## Inspecting the Learned Distribution

### Check 1: Sampling

Sample a single $\theta$ for many different conditions $x \in [0., 1.]$:

$$
\theta \sim \text{MDN}(\alpha(x), \mu(x), \sigma(x))
$$

In [ ]:
samples = []
x_test = torch.linspace(-0.1, 1.1, 500).unsqueeze(1)

for single_x in tqdm(x_test):
    # compute the mixture components
    weights, means, variances = mixture_density_net.get_mixture_components(
        single_x.unsqueeze(1)
    )
    # sample form mixture of Gaussians
    sample = mdn.mog_sample(weights, means, variances)

    samples.append(sample)

samples = torch.cat(samples).detach().numpy()


### Check 2: Visualizing the conditional density at a particular $x$

We fix one particular $x$, and want to visualize the conditional PDF given that $x$: 

$$
p(\theta \mid x = 0.5)
$$ 

To visualize the PDF, we evaluate it along the values $\theta$ can take, e.g., 100 evenly spaced values between 0 and 1. 

**Task:** Use the trained posterior approximator to compute the conditional distribution over $\theta$, given an observation, e.g. $x_{\text{obs}} = 0.5$.

Therefore, obtain the parameters for the Gaussian mixture using the `.get_mixture_components(..)` method of the density estimator object defined above. Then, use the `mog_log_prob(..)` function of the `mdn` module (inside a loop) to evaluate the density of all $\theta_i$ along the axis of $x_{\text{obs}}$.

In order to stick with following cells, please store the obtained densities in a tensor called `demo_probs`.

Feel free to use the following structure.

```python
# chose value to condition on & compute the mixture components
x_demo = torch.as_tensor([[..]])  # todo: define the observation
logits, means, variances = # todo: obtain mixture parameters 

demo_probs = []

for t_demo in tqdm(torch.linspace(-0.1, 1.0, 100)):
    # todo: compute probability of different values of theta for the current parameterization
    # ..

demo_probs = torch.stack(demo_probs).detach().exp()
```


In [ ]:
# Solution task 3.
# %load -r 61-74 ./solutions/solutions_nb_02.py

In [ ]:
plt.subplots(1, 1, figsize=(8, 4))

plt.plot(x[::15], theta[::15], "go", alpha=0.25, label="data")
plt.plot(x_test, samples, "mo", alpha=0.5, label="samples")
# Note: we scale down the probs only for visualization.
plt.plot(probs.numpy() * 0.1 + x_o.numpy(), theta_test, linewidth=2, color="black", 
         label=r"$p(\theta \mid x=0.5)$")
plt.axvline(x_o.data[0, 0], color="black", linestyle="--")

plt.xlabel("$x$", fontsize=16)
plt.ylabel("$\\theta$", fontsize=16)
plt.legend();

## Summary: Conditional density estimation with MDNs

### What are the benefits? 

- The MDN learns a functional relationship between $x$ and $\theta$.
- This approach can exploit continuities in the parameter space.
- Amortization: once trained, the MDN can be used for many different $x$.

### What are the limitations?

- Requires setting of hyperparameters: e.g. components
- Training often does not converge perfectly. 
- In practice, high-dimensional distributions might not be captured perfectly
- Normalizing flows, VAEs, and GANs are more flexible. However, Mixtures of Gaussians often allow for useful computations in closed form and are still sometimes used in practice.


## Learning goals

1) What is density estimation? 
2) What is conditional density estimation? 
3) What are mixture density networks?
4) What is the connection to simulation-based inference? 

## Anything unclear? Please ask! 

## Acknowledgments

- Material is based on the [MLColab training](https://mlcolab.org/simulation-based-inference-for-scientific-discovery) on SBI <br/>
- Parts of the code from [Mike Dusenberry](https://mikedusenberry.com/mixture-density-networks)  <br/>
- Code of MDNs based on Conor Durkan's `lfi` package.  <br/>
- Bishop, 1994  <br/>
- [MDN graphic](https://towardsdatascience.com/a-hitchhikers-guide-to-mixture-density-networks-76b435826cca)  
- Pedro Gonçalves et al. for figure.

<img src="_static/images/aai-institute-cover.png" alt="Snow" style="width:100%;">
<div class="md-slide title">Thank you for the attention!</div>